## LMFunction example

In [3]:
from llm_programs.programs import LMFunction, AutoPrompter, Gemini, YesNoPredicateParser

In [27]:
# There is only *one* placeholder in the prompt template, so AutoPrompter will result in a *unary* function.
template = '''Is the following statement true? \'{statement}\' Begin your answer with "Yes" or "No".'''
prompter = AutoPrompter(template)

# NOTE: You must set GEMINI_API_KEY=... in the .env file at the repo root
engine = Gemini(debug=True)

# The parser will return a boolean value (True for Yes, False for No)
parser = YesNoPredicateParser()

# We have a Gemini-powered unary predicate function! 
factcheck = LMFunction(prompter=prompter, engine=engine, parser=parser)

In [28]:
factcheck("The sky is blue.")  # True

==== Prompt ==== (n_llm_calls = 1)
Is the following statement true? 'The sky is blue.' Begin your answer with "Yes" or "No".
=== Response ===
Yes





True

In [ ]:
statements = [
    "Bats are the only mammals capable of sustained flight.", # True
    "A group of flamingos is called a flamboyance.", # True
    "Koalas are bears.", # False
    "Penguins live in the Arctic.", # False
]

In [29]:
list(filter(factcheck, statements))

==== Prompt ==== (n_llm_calls = 2)
Is the following statement true? 'Bats are the only mammals capable of sustained flight.' Begin your answer with "Yes" or "No".
=== Response ===
Yes



==== Prompt ==== (n_llm_calls = 3)
Is the following statement true? 'A group of flamingos is called a flamboyance.' Begin your answer with "Yes" or "No".
=== Response ===
Yes



==== Prompt ==== (n_llm_calls = 4)
Is the following statement true? 'Koalas are bears.' Begin your answer with "Yes" or "No".
=== Response ===
No.



==== Prompt ==== (n_llm_calls = 5)
Is the following statement true? 'Penguins live in the Arctic.' Begin your answer with "Yes" or "No".
=== Response ===
No.





['Bats are the only mammals capable of sustained flight.',
 'A group of flamingos is called a flamboyance.']

In [ ]:
prompt = "Invent a new plausible realistic fact about penguins that sounds true but is actually subtly false. State it in one very brief sentence."

generate = LMFunction(
    prompter=AutoPrompter(prompt),
    engine=engine,
)

In [26]:
generate()

==== Prompt ==== (n_llm_calls = 64)
Invent a new plausible realistic fact about penguins that sounds true but is actually false. State it to one very brief sentence.
=== Response ===
Emperor penguins are born with a temporary layer of reflective scales to camouflage them amongst the ice during their first weeks of life.





'Emperor penguins are born with a temporary layer of reflective scales to camouflage them amongst the ice during their first weeks of life.\n'

In [ ]:
# Search for a bogus fact that can fool Gemini
fact = generate()
while not factcheck(fact):
    fact = generate()
fact

==== Prompt ==== (n_llm_calls = 65)
Invent a new plausible realistic fact about penguins that sounds true but is actually false. State it to one very brief sentence.
=== Response ===
Penguins have a rudimentary form of echolocation to navigate murky waters during feeding.



==== Prompt ==== (n_llm_calls = 6)
Is the following statement true? 'Penguins have a rudimentary form of echolocation to navigate murky waters during feeding.
' Begin your answer with "Yes" or "No".
=== Response ===
No.



==== Prompt ==== (n_llm_calls = 66)
Invent a new plausible realistic fact about penguins that sounds true but is actually false. State it to one very brief sentence.
=== Response ===
Penguin feathers are mildly bioluminescent, creating a faint glow that helps them locate each other in the dark ocean depths.



==== Prompt ==== (n_llm_calls = 7)
Is the following statement true? 'Penguin feathers are mildly bioluminescent, creating a faint glow that helps them locate each other in the dark ocean de

## Synthetic contract generation and redaction

In [36]:
from llm_programs.tasks.contract_generation.redaction.generate import generate_contract_aux
from llm_programs.programs import Gemini
from llm_programs.tasks.redaction.eval import one_call_m_prompts_fn

In [ ]:
engine = Gemini(debug=True)
info, descr, contract = generate_contract_aux(engine)

==== Prompt ==== (n_llm_calls = 1)
Give a few scientific & publishing institutions *and* their abbreviations. Output only one per line. Do not output any other text.

EXAMPLES:
Technische Universiteit Delft
TU Delft
TUD
Elsevier

=== Response ===
Massachusetts Institute of Technology
MIT
Springer Nature
SN
American Association for the Advancement of Science
AAAS
Wiley
Institute of Electrical and Electronics Engineers
IEEE
National Academy of Sciences
NAS
Public Library of Science
PLOS
Oxford University Press
OUP
Cambridge University Press
CUP



==== Prompt ==== (n_llm_calls = 2)
Generate a few random names. Output only one per line. Do not output any other text.

EXAMPLES:
Michael K. Scottson
Hjalmar Makela
B. Aaronson

=== Response ===
Eliza Maxwell
Jean-Pierre Dubois
Ingrid Bjornstad
Kimiko Tanaka
Jamal Al-Fayed



==== Prompt ==== (n_llm_calls = 3)
Generate a few random telephone numbers. Output only one per line. Do not output any other text.

EXAMPLES:
+1 (555) 787-7890
+41 7 784

In [43]:
keywords = one_call_m_prompts_fn(contract)

==== Prompt ==== (n_llm_calls = 12)
From the provided document identify all the full names of the participating institutions, and their abbreviations. Output each institution full name or abbreviation on a new line. If none are found, output `None`. Please do not output additional text.

For example:
```
American Biophysica1 Union
ABU
```

=== BEGIN DOCUMENT EXTRACT ===

**ACADEMIC CONTENT LICENSE AGREEMENT**

This Academic Content License Agreement (the "Agreement") is made and entered into as of June 03, 2010, by and between Massachusetts Institute of Technology, hereinafter referred to as "MIT", and Springer Nature, hereinafter referred to as "SN". This Agreement amends and restates the agreement initially drafted on 2/14/2001.

**1. Preamble / Recitals**

WHEREAS, SN is a leading publisher of academic books, journals, and online resources; and

WHEREAS, MIT desires to obtain a license to access SN's digital library for the benefit of its students, faculty, and staff; and

WHEREAS, 

In [44]:
new_contract = contract
for keyword in keywords:
    new_contract = new_contract.replace(keyword, "[REDACTED]")
print(new_contract)

**ACADEMIC CONTENT LICENSE AGREEMENT**

This Academic Content License Agreement (the "Agreement") is made and entered into as of [REDACTED], by and between [REDACTED], hereinafter referred to as "[REDACTED]", and [REDACTED], hereinafter referred to as "[REDACTED]". This Agreement amends and restates the agreement initially drafted on [REDACTED].

**1. Preamble / Recitals**

WHEREAS, [REDACTED] is a leading publisher of academic books, journals, and online resources; and

WHEREAS, [REDACTED] desires to obtain a license to access [REDACTED]'s digital library for the benefit of its students, faculty, and staff; and

WHEREAS, [REDACTED] is willing to grant such a license to [REDACTED] subject to the terms and conditions set forth herein;

NOW, THEREFORE, in consideration of the mutual covenants contained herein, the parties agree as follows:

**2. Definitions**

*   **"Licensed Materials"** shall mean the collection of e-books and journal articles related to engineering and scientific rese